In [1]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 4.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 53.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 61.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 81.2 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 3.4 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=0cb8b2404c23250999ea5d48248f0f3bb3b93f0d701b8b51229f4dd109964911
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
  Attempting uninstall: tzlocal
    Found existing installation: tzlocal 5.0.1
    Uninstalling tzlocal-5.0.1:
      Successfully uninstalled tzlocal-5.0.1
  A

# Libraries & Functions

In [2]:
import pandas as pd 
import numpy as np
from tqdm import tqdm

In [3]:
from simpletransformers.classification import ClassificationModel
import torch

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder

def calculate_scores(y_test, y_pred, average = "binary"):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average = average)
    recall = recall_score(y_test, y_pred, average = average)
    f1 = f1_score(y_test, y_pred, average = average)
    return [accuracy, precision, recall, f1]

In [5]:
model_names = ["distilbert", "distilbert", "debertav2", "electra"]
checkpoint_names = ["ViktorDo/EcoBERT-Pretrained", "distilbert-base-uncased", "microsoft/deberta-v3-base", "google/electra-base-discriminator"]

# Input Data

In [6]:
raw_datasets = dict()

In [7]:
description_threshold = 100

## POWO Dataset

In [8]:
working_dir = "..//input//powo-family//" 

df_POWO_Fam =  pd.read_excel(working_dir + "POWO_Family.xlsx")
df_POWO_Fam_Preproc = df_POWO_Fam.drop_duplicates(subset = ["BERT_description"])
df_POWO_Fam_Preproc = df_POWO_Fam_Preproc[df_POWO_Fam_Preproc["BERT_description"].apply(lambda x: len(x.split(" ")))>10]

POWO_Filter = df_POWO_Fam_Preproc["family"].value_counts().keys().values[:10]
df_POWO_Fam_Preproc = df_POWO_Fam_Preproc[df_POWO_Fam_Preproc["family"].apply(lambda x: x in POWO_Filter)].groupby('family', group_keys=False).apply(lambda x: x.sample(500, random_state = 42))
raw_datasets["POWO"] = df_POWO_Fam_Preproc

## WIKI Dataset

In [9]:
def fix_WIKI(name, description):
    for n in name.split(" "):
        description = str(description).replace(n.lower(), "")
    return description.strip()

In [10]:
working_dir = "..//input//wiki-family//" 

df_WIKI_Fam =  pd.read_excel(working_dir + "WIKI_Family.xlsx")
df_WIKI_Fam_Preproc = df_WIKI_Fam.drop_duplicates(subset = ["BERT_description"])
df_WIKI_Fam_Preproc["BERT_description"] = df_WIKI_Fam_Preproc[["name", "BERT_description"]].apply(lambda x: fix_WIKI(x[0], x[1]), axis = 1)
df_WIKI_Fam_Preproc = df_WIKI_Fam_Preproc[df_WIKI_Fam_Preproc["BERT_description"].apply(lambda x: len(str(x).split(" ")))>10]

WIKI_Filter = df_WIKI_Fam_Preproc["family"].value_counts().keys().values[:10]
df_WIKI_Fam_Preproc = df_WIKI_Fam_Preproc[df_WIKI_Fam_Preproc["family"].apply(lambda x: x in WIKI_Filter)].groupby('family', group_keys=False).apply(lambda x: x.sample(500, random_state = 42))
raw_datasets["WIKI"] = df_WIKI_Fam_Preproc

/tmp/ipykernel_29/1233968646.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_WIKI_Fam_Preproc["BERT_description"] = df_WIKI_Fam_Preproc[["name", "BERT_description"]].apply(lambda x: fix_WIKI(x[0], x[1]), axis = 1)


## Preprocess Datasets

In [11]:
preprocessed_dataset_dict = {}
for dataset_name in list(raw_datasets.keys()):
    labelencoder = LabelEncoder()
    raw_datasets[dataset_name]["family_encoded"] = labelencoder.fit_transform(raw_datasets[dataset_name]["family"])

    indices_train, indices_test \
        = train_test_split(raw_datasets[dataset_name].index.values, test_size=0.25, random_state=42)
    
    
    df_train = raw_datasets[dataset_name].loc[indices_train, ["BERT_description", "family_encoded"]]
    df_train.columns = ["text", "labels"]
    df_test = raw_datasets[dataset_name].loc[indices_test, ["BERT_description", "family_encoded"]]
    df_test.columns = ["text", "labels"]

    preprocessed_dataset_dict[dataset_name, "train"] = df_train
    preprocessed_dataset_dict[dataset_name, "validation"] = df_test

# Model Training & Evaluation

In [12]:
if torch.cuda.is_available():  # Tell PyTorch to use the GPU. 
    device = torch.device("cuda") 
    print('There are %d GPU(s) available.' % torch.cuda.device_count()) 
    print('We will use the GPU:', torch.cuda.get_device_name(0)) # If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
    
import gc
torch.cuda.empty_cache()
gc.collect()

There are 2 GPU(s) available.
We will use the GPU: Tesla T4


130

In [13]:
results_list = []

for model_name, model_checkpoint in zip(model_names[:], checkpoint_names[:]):
    for dataset_name in list(raw_datasets.keys())[:]:

        print(model_checkpoint, dataset_name)
        
        if(model_checkpoint == "ViktorDo/EcoBERT-Pretrained"):
            model = ClassificationModel(
                model_name,
                model_checkpoint,
                tokenizer_name = "distilbert-base-uncased",
                num_labels = 10, #preprocessed_dataset_dict[dataset_name, "train"]["labels"].nunique(),
                args = {"num_train_epochs": 3, "train_batch_size":8, "eval_batch_size":8, "reprocess_input_data": True, "overwrite_output_dir": True, "save_model_every_epoch": False, "save_eval_checkpoints": False, "max_seq_length": 512}, #"weight_decay": 0.01, "learning_rate": 2e-5, 
            )
        else:
            model = ClassificationModel(
                model_name,
                model_checkpoint,
                num_labels = 10, #preprocessed_dataset_dict[dataset_name, "train"]["labels"].nunique(),
                args = {"num_train_epochs": 3, "train_batch_size":8, "eval_batch_size":8, "reprocess_input_data": True, "overwrite_output_dir": True, "save_model_every_epoch": False, "save_eval_checkpoints": False, "max_seq_length": 512}, #"weight_decay": 0.01, "learning_rate": 2e-5, 
            )
        # Train the model
        model.train_model(preprocessed_dataset_dict[dataset_name, "train"])

        # Evaluate the model
        result, model_outputs, wrong_predictions = model.eval_model(preprocessed_dataset_dict[dataset_name, "validation"])
        preprocessed_dataset_dict[dataset_name, "validation"]["prediction"] = np.argmax(model_outputs, axis=1)
        results = calculate_scores(preprocessed_dataset_dict[dataset_name, "validation"]["labels"], preprocessed_dataset_dict[dataset_name, "validation"]["prediction"], average = "macro")
        results_list.append([dataset_name, "Family"] + results + [model_name])

        torch.cuda.empty_cache()
        gc.collect()


df_results = pd.DataFrame(results_list, columns=["Dataset", "Trait", "Accuracy", "Precision", "Recall", "F1-Score", "Model"])

ViktorDo/EcoBERT-Pretrained POWO


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at ViktorDo/EcoBERT-Pretrained and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/3750 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/469 [00:00<?, ?it/s]

...

Running Epoch 0 of 3:   0%|          | 0/469 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/469 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/1250 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/157 [00:00<?, ?it/s]

In [14]:
df_results = pd.DataFrame(results_list, columns=["Dataset", "Trait", "Accuracy", "Precision", "Recall", "F1-Score", "Model"])

In [15]:
df_results

,Dataset,Trait,Accuracy,Precision,Recall,F1-Score,Model
0,POWO,Family,0.9648,0.965422,0.965381,0.965209,distilbert
1,WIKI,Family,0.9712,0.970690,0.970147,0.970250,distilbert
2,POWO,Family,0.9680,0.968400,0.968147,0.968233,distilbert
3,WIKI,Family,0.9712,0.970701,0.970070,0.970276,distilbert
4,POWO,Family,0.9480,0.949728,0.948704,0.948944,debertav2
5,WIKI,Family,0.9656,0.965516,0.964464,0.964734,debertav2
6,POWO,Family,0.9640,0.965100,0.964041,0.964369,electra
7,WIKI,Family,0.9712,0.970611,0.970563,0.970299,electra


In [16]:
df_results.to_excel("FamilyClassification_Encoder_Results.xlsx", index = False)

In [17]:
df_results

,Dataset,Trait,Accuracy,Precision,Recall,F1-Score,Model
0,POWO,Family,0.9648,0.965422,0.965381,0.965209,distilbert
1,WIKI,Family,0.9712,0.970690,0.970147,0.970250,distilbert
2,POWO,Family,0.9680,0.968400,0.968147,0.968233,distilbert
3,WIKI,Family,0.9712,0.970701,0.970070,0.970276,distilbert
4,POWO,Family,0.9480,0.949728,0.948704,0.948944,debertav2
5,WIKI,Family,0.9656,0.965516,0.964464,0.964734,debertav2
6,POWO,Family,0.9640,0.965100,0.964041,0.964369,electra
7,WIKI,Family,0.9712,0.970611,0.970563,0.970299,electra
